In [3]:
# !pip install googletrans==4.0.0rc1
# pip install pyldavis
# !pip install easynmt
# nltk.download('omw-1.4')
pd.set_option("max_colwidth", None)

In [2]:
import gensim
from gensim.utils import simple_preprocess
import nltk.corpus
import nltk
import pandas as pd
import gensim.corpora
from langdetect import detect
from googletrans import Translator
from nltk.corpus import stopwords
import numpy
import time

#library that contains punctuation
import string
string.punctuation
from nltk.stem import WordNetLemmatizer

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

translater = Translator()

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [246]:
data = pd.read_csv("combined_profile.csv")
data['translated_occupation'].nunique()

715

In [247]:
data['translated_occupation']= data['translated_occupation'].apply(lambda x: x.lower())
data['translated_occupation'].nunique()

523

In [248]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#storing the puntuation free text
data['translated_occupation']= data['translated_occupation'].apply(lambda x:remove_punctuation(x))
data['translated_occupation'].nunique()

518

In [249]:
data['translated_occupation'].apply(lambda x:x.strip())

0             student
1             student
2             student
3             student
4             student
            ...      
7929         engineer
7930    self employed
7931         military
7932     selfemployed
7933    self employed
Name: translated_occupation, Length: 7934, dtype: object

In [250]:
#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = wordnet_lemmatizer.lemmatize(text)
    return lemm_text

data['translated_occupation_lem']=data['translated_occupation'].apply(lambda x:lemmatizer(x))


In [443]:
data = pd.read_csv("data_cat_draft.csv")
len(data)

7934

In [5]:
data['translated_occupation_lem'].value_counts()

student               541
engineer              493
self employed         427
military              389
retired               267
                     ... 
roofer                  2
professor               1
gerente comercial       1
commercial manager      1
salesmarketing          1
Name: translated_occupation_lem, Length: 513, dtype: int64

In [6]:
data['translated_occupation_lem'].unique()

array(['student', 'study', 'manager', 'law student', 'unemployed',
       'educator', 'waitress', 'manicurist', 'university', 'baby sister',
       'independent', 'to study', 'secretary', 'plumber', 'contador',
       'salesman', 'enterprising', 'studying', 'worked', 'nurse', 'clerk',
       'college student', 'construction', 'baker', 'self employed',
       'housewife', 'merchant', 'teacher', 'engineer',
       'business administrator', 'technician', 'office', 'barber',
       'study and work', 'beautician', 'journalist', 'customer service',
       'landscaping', 'customer support', 'painter', 'chef', 'barista',
       'soccer player', 'work', 'stylist', 'building', 'musician',
       'electrician', 'lawyer', 'universidad', 'nursing', 'military',
       'typist', 'cashier', 'conductor', 'security guard', 'cocina',
       'management', 'entrepreneur', 'male nurse', 'music producer',
       'transport', 'uber', 'assistant', 'paralegal', 'cook',
       'accounting', 'law', 'freelance', '

In [433]:
for index, row in data.iterrows():
    if "system" == row['translated_occupation_lem']:
        data.at[index , 'translated_occupation_category'] = "tech"

In [440]:
# data[data['translated_occupation']=="college student"]
data['translated_occupation_category'].fillna('others',inplace=True)

In [442]:
data['translated_occupation_category'].value_counts().sum()

7934

In [444]:
data.to_csv("data_cat_draft.csv")

In [ ]:
data

In [40]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [79]:
odata_words = list(sent_to_words(occupation))


[['retired']]


In [76]:
def flatten(lis):
    flatList = []
    # Iterate with outer list
    for element in lis:
        if type(element) is list:
            # Check if type is list than iterate through the sublist
            for item in element:
                flatList.append(item)
        else:
            flatList.append(element)
    return flatList

In [82]:
odata_flatten = flatten(odata_words)
odata_flatten
# odata_words
print(odata_flatten[:30])

['retired', 'estudiante', 'construction', 'retired', 'social', 'worker', 'retired', 'executive', 'journalist', 'designer', 'retired', 'chef', 'entrepreneur', 'student', 'retired', 'medical', 'service', 'author', 'engineer', 'retired', 'lawyer', 'teacher', 'care', 'worker', 'cobranza', 'chofer', 'lawyer', 'disabled', 'it', 'technician']


In [90]:
import gensim.corpora as corpora
# # Create Dictionary
id2word = corpora.Dictionary(odata_words)
# Create Corpus
texts = odata_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])


[(0, 1)]


In [91]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.108*"business" + 0.078*"engineer" + 0.061*"nurse" + 0.054*"retired" + '
  '0.033*"abogado" + 0.028*"owner" + 0.024*"student" + 0.023*"manager" + '
  '0.018*"social" + 0.017*"worker"'),
 (1,
  '0.195*"self" + 0.191*"employed" + 0.055*"student" + 0.045*"military" + '
  '0.038*"estudiante" + 0.019*"abogado" + 0.018*"driver" + 0.018*"ventas" + '
  '0.013*"docente" + 0.012*"truck"'),
 (2,
  '0.094*"ingeniero" + 0.066*"independiente" + 0.050*"tecnico" + '
  '0.048*"engineering" + 0.026*"mecanico" + 0.026*"contador" + 0.024*"de" + '
  '0.022*"engineer" + 0.021*"nurse" + 0.017*"comerciante"'),
 (3,
  '0.072*"student" + 0.068*"construction" + 0.048*"business" + '
  '0.035*"estudiante" + 0.033*"doctor" + 0.033*"profesor" + 0.032*"military" + '
  '0.030*"enfermera" + 0.028*"chofer" + 0.026*"administrador"'),
 (4,
  '0.105*"retired" + 0.070*"student" + 0.061*"ingeniero" + 0.058*"estudiante" '
  '+ 0.049*"docente" + 0.043*"comerciante" + 0.024*"nurse" + '
  '0.023*"independiente" + 0.021*

In [127]:
data['occupation'].groupby(occupation).count()

occupation
 Accountant         5
 Administration     5
 Administrative     6
 Architect         11
 Architecture       3
                   ..
vendedor            5
ventas              3
welder              3
Área de salud       2
Преподаватель       2
Name: occupation, Length: 1016, dtype: int64